In [1]:
import nltk
import pandas as pd
import ast
import re
import numpy as np

import string 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

import unidecode
import nltk.corpus
from gensim.models import Word2Vec

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import config

[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>


In [4]:
df = pd.read_csv("Cleaned_Indian_Food_Dataset.csv")

In [5]:
df.rename(columns = {'Cleaned-Ingredients':'ingredients'}, inplace = True)
for i in range(len(df["ingredients"])):
    df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
    df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
    df["ingredients"][i] = df["ingredients"][i].split(',')

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_11555/1364071601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_11555/1364071601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_11555/1364071601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [6]:
df

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"[salt, amchur , karela , red chilli powder, gr...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"[tomato, salt, chickpea lentils, green chilli,...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"[salt, rice vermicelli noodles , asafoetida , ...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"[tomato, salt, ginger, sorrel leaves , fennel ...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"[tomato, salt, ginger, red chillies, curry, as...",https://www.archanaskitchen.com/images/archana...,12
...,...,...,...,...,...,...,...,...,...
5933,Saffron Paneer Peda Recipe,"2 teaspoons Rose water,1/2 teaspoon Ghee,2 cup...",20,Indian,To begin making the Saffron Paneer Peda recipe...,https://www.archanaskitchen.com/saffron-paneer...,"[saffron strands generous, ghee, paneer, rose ...",https://www.archanaskitchen.com/images/archana...,7
5934,Italian Arancini Rice Balls Recipe With Delici...,"4 sprig Basil leaves - finely chopped,2 cups W...",100,Italian Recipes,To begin making the Italian Arancini Rice Ball...,https://www.archanaskitchen.com/italian-aranci...,"[salt, gorgonzola cheese cubes, wheat bread cr...",https://www.archanaskitchen.com/images/archana...,14
5935,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"3/4 cup Sugar,2 cup Milk - vegans can substitu...",35,Indian,"To begin making Quinoa Phirnee Recipe, place a...",https://www.archanaskitchen.com/quinoa-phirnee...,"[ghee, milk, saffron strands, sugar, cardamom ...",https://www.archanaskitchen.com/images/archana...,8
5936,Ullikadala Pulusu Recipe | Spring Onion Curry,"1 pinch Turmeric powder (Haldi),1/2 teaspoon M...",15,Andhra,To begin making Ullikadala Pulusu Recipe | Spr...,https://www.archanaskitchen.com/ullikadala-pul...,"[salt, tamarind water, mustard seeds, asafoeti...",https://www.archanaskitchen.com/images/archana...,13


In [7]:
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       pass
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    remove_punctuations = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for each_item in ingredients:
        each_item.translate(remove_punctuations)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', each_item)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list

In [8]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted
  
# calculate average length of each document 
def get_window(corpus):
    lengths = [len(doc) for doc in corpus]
    avg_len = float(sum(lengths)) / len(lengths)
    return round(avg_len)

if __name__ == "__main__":
    # load in data
    # parse the ingredients for each recipe
    df['parsed'] = df.ingredients.apply(ingredient_parser)
    # get corpus
    corpus = get_and_sort_corpus(df)
    print(f"Length of corpus: {len(corpus)}")
    # train and save CBOW Word2Vec model

Length of corpus: 5938


In [14]:
from fuzzywuzzy import fuzz

/opt/homebrew/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
non_veg_keywords = ["chicken", "beef", "pork", "fish", "meat", "mutton"]
i=0
cat = []
for row in df['parsed']:
    count = False
    for item1 in row:
        for item in non_veg_keywords:
            score = fuzz.partial_token_set_ratio(item1, item)
            if score == 100:
                count = True
                continue
    if count:
        cat.append('non-veg')
    else:
        cat.append('veg')


In [16]:
df['category'] = cat

In [17]:
df.to_csv('data.csv', index=False) 

In [18]:
list = []
for i in df['parsed']:
    list = list + i
unique = set(list)

len(unique)

4164